In [1]:
import os
import time
import random
import numpy as np
import logging

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.multiprocessing as mp
import torch.distributed as dist
from tensorboardX import SummaryWriter

from MinkowskiEngine import SparseTensor
from util import config
from util.util import AverageMeter, intersectionAndUnionGPU, \
    poly_learning_rate, save_checkpoint, \
    export_pointcloud, get_palette, convert_labels_with_palette, extract_clip_feature
from dataset.label_constants import *
from dataset.feature_loader import FusedFeatureLoader, collation_fn
from dataset.point_loader import Point3DLoader, collation_fn_eval_all
from tqdm import tqdm


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# changed code here since we don't use argparser in notebook
from models.mink_unet import mink_unet as model3D

def constructor3d(**kwargs):
    model = model3D(**kwargs)
    return model


class DisNet(nn.Module):
    '''3D Sparse UNet for Distillation.'''
    def __init__(self):
        super(DisNet, self).__init__()
        last_dim = 768

        # MinkowskiNet for 3D point clouds
        net3d = constructor3d(in_channels=3, out_channels=last_dim, D=3, arch="MinkUNet18A")
        self.net3d = net3d

    def forward(self, sparse_3d):
        '''Forward method.'''
        return self.net3d(sparse_3d)


In [3]:
def worker_init_fn(worker_id):
    '''Worker initialization.'''
    random.seed(time.time() + worker_id)
    
def get_logger():
    '''Define logger.'''
    logger_name = "main-logger"
    logger_in = logging.getLogger(logger_name)
    logger_in.setLevel(logging.DEBUG)
    handler = logging.StreamHandler()
    fmt = "[%(asctime)s %(filename)s line %(lineno)d] %(message)s"
    handler.setFormatter(logging.Formatter(fmt))
    logger_in.addHandler(handler)
    return logger_in


In [4]:
def obtain_text_features_and_palette():
    '''obtain the CLIP text feature and palette.'''

    
    labelset = list(SCANNET_LABELS_20)
    labelset[-1] = 'other'
    palette = get_palette()
    dataset_name = 'scannet'

    if not os.path.exists('saved_text_embeddings'):
        os.makedirs('saved_text_embeddings')
    model_name="ViT-L/14@336px"
    postfix = '_768' # the dimension of CLIP features is 768

    clip_file_name = 'saved_text_embeddings/clip_{}_labels{}.pt'.format(dataset_name, postfix)

    try: # try to load the pre-saved embedding first
        logger.info('Load pre-computed embeddings from {}'.format(clip_file_name))
        text_features = torch.load(clip_file_name).cuda()
    except: # extract CLIP text features and save them
        text_features = extract_clip_feature(labelset, model_name=model_name)
        torch.save(text_features, clip_file_name)

    return text_features, palette

In [5]:
def distill(train_loader, model, optimizer, epoch):
    '''Distillation pipeline.'''

    torch.backends.cudnn.enabled = True
    batch_time = AverageMeter()
    data_time = AverageMeter()

    loss_meter = AverageMeter()

    model.train()
    end = time.time()
    max_iter = EPOCHS * len(train_loader)

    text_features, palette = obtain_text_features_and_palette()

    # start the distillation process
    for i, batch_data in enumerate(train_loader):
        print("inside for loop")
        data_time.update(time.time() - end)

        (coords, feat, label_3d, feat_3d, mask) = batch_data
        print(coords)
        coords[:, 1:4] += (torch.rand(3) * 100).type_as(coords)
        sinput = SparseTensor(
            feat.cuda(non_blocking=True), coords.cuda(non_blocking=True))
        feat_3d, mask = feat_3d.cuda(
            non_blocking=True), mask.cuda(non_blocking=True)

        output_3d = model(sinput)
        output_3d = output_3d[mask]

        loss = (1 - torch.nn.CosineSimilarity()(output_3d, feat_3d)).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_meter.update(loss.item(), BATCH_SIZE)
        batch_time.update(time.time() - end)

        # adjust learning rate
        current_iter = epoch * len(train_loader) + i + 1
        current_lr = poly_learning_rate(
            BASE_LR, current_iter, max_iter, power=POWER)

        for index in range(0, INDEX_SPLIT):
            optimizer.param_groups[index]['lr'] = current_lr
        for index in range(INDEX_SPLIT, len(optimizer.param_groups)):
            optimizer.param_groups[index]['lr'] = current_lr * 10

        # calculate remain time
        remain_iter = max_iter - current_iter
        remain_time = remain_iter * batch_time.avg
        t_m, t_s = divmod(remain_time, 60)
        t_h, t_m = divmod(t_m, 60)
        remain_time = '{:02d}:{:02d}:{:02d}'.format(
            int(t_h), int(t_m), int(t_s))

        if (i + 1) % PRINT_FREQ == 0 :
            logger.info('Epoch: [{}/{}][{}/{}] '
                        'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                        'Batch {batch_time.val:.3f} ({batch_time.avg:.3f}) '
                        'Remain {remain_time} '
                        'Loss {loss_meter.val:.4f} '.format(epoch + 1, EPOCHS, i + 1, len(train_loader),
                                                            batch_time = batch_time, data_time = data_time,
                                                            remain_time = remain_time,
                                                            loss_meter = loss_meter))
        
        writer.add_scalar('loss_train_batch', loss_meter.val, current_iter)
        writer.add_scalar('learning_rate', current_lr, current_iter)

        end = time.time()

    mask_first = (coords[mask][:, 0] == 0)
    output_3d = output_3d[mask_first]
    feat_3d = feat_3d[mask_first]
    logits_pred = output_3d.half() @ text_features.t()
    logits_img = feat_3d.half() @ text_features.t()
    logits_pred = torch.max(logits_pred, 1)[1].cpu().numpy()
    logits_img = torch.max(logits_img, 1)[1].cpu().numpy()
    mask = mask.cpu().numpy()
    logits_gt = label_3d.numpy()[mask][mask_first.cpu().numpy()]
    logits_gt[logits_gt == 255] = CLASSES

    pcl = coords[:, 1:].cpu().numpy()

    seg_label_color = convert_labels_with_palette(
        logits_img, palette)
    pred_label_color = convert_labels_with_palette(
        logits_pred, palette)
    gt_label_color = convert_labels_with_palette(
        logits_gt, palette)
    pcl_part = pcl[mask][mask_first.cpu().numpy()]

    export_pointcloud(os.path.join(SAVE_PATH, 'result', 'last', 'openseg_{}.ply'.format(
        epoch)), pcl_part, colors = seg_label_color)
    export_pointcloud(os.path.join(SAVE_PATH, 'result', 'last',
                        'pred_{}.ply'.format(epoch)), pcl_part, colors = pred_label_color)
    export_pointcloud(os.path.join(SAVE_PATH, 'result', 'last',
                        'gt_{}.ply'.format(epoch)), pcl_part, colors = gt_label_color)

    return loss_meter.avg


In [6]:
best_iou = 0.0
manual_seed = 1453
random.seed(manual_seed)
np.random.seed(manual_seed)
torch.manual_seed(manual_seed)
torch.cuda.manual_seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

In [7]:
DATA_ROOT = "/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_3d"
DATA_ROOT_FUSED = "/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_fused_features"
SAVE_PATH = "/mnt/project/AT3DCV_Data/experiments"
VOXEL_SIZE = 0.02
AUG = True
USE_SHM = False
LOOP = 5
INPUT_COLOR = False
INDEX_SPLIT = 0
BATCH_SIZE = 1
START_EPOCH = 0 
EPOCHS = 100
BASE_LR = 0.0001
POWER = 0.9
PRINT_FREQ = 10
CLASSES = 20 # Scannet

In [8]:
model = DisNet()
optimizer = torch.optim.Adam(model.parameters(), lr = BASE_LR)
model = model.cuda()

In [9]:
global logger, writer
logger = get_logger()
writer = SummaryWriter(SAVE_PATH)
#logger.info(args)
logger.info("=> creating model ...")

[2023-07-01 10:03:52,169 1590180057.py line 5] => creating model ...


In [10]:
train_data = FusedFeatureLoader(datapath_prefix = DATA_ROOT,
                                datapath_prefix_feat = DATA_ROOT_FUSED,
                                voxel_size = VOXEL_SIZE,
                                split = 'train', aug = AUG,
                                memcache_init = USE_SHM, loop = LOOP,
                                input_color = INPUT_COLOR
                                )
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE,
                                            shuffle = True,
                                            num_workers = 4, pin_memory = True,
                                            sampler = None,
                                            drop_last = True, collate_fn = collation_fn,
                                            worker_init_fn = worker_init_fn)

In [11]:
len(train_loader)

5

In [12]:
for i, batch_data in enumerate(train_loader):
    print(i)

0
1
2
3
4


In [20]:
sample = train_data.__getitem__(12) 
"""
1) coords: voxel coordinates
2) feats: colors (all 1s since we set INPUT_COLOR = False)
3) labels: point labels, 255 is unknown
4) feat_3d: fused features in CLIP space [768]
5) mask: mask for 3D points
""" 

'\n1) coords: voxel coordinates\n2) feats: colors (all 1s since we set INPUT_COLOR = False)\n3) labels: point labels, 255 is unknown\n4) feat_3d: fused features in CLIP space [768]\n5) mask: mask for 3D points\n'

In [ ]:
for epoch in range(START_EPOCH, EPOCHS):
    loss_train = distill(train_loader, model, optimizer, epoch)
    epoch_log = epoch + 1
    writer.add_scalar('loss_train', loss_train, epoch_log)
    writer.close()
    logger.info('==>Training done!\nBest Iou: %.3f' % (best_iou))

[2023-07-01 10:04:05,226 1544885347.py line 18] Load pre-computed embeddings from saved_text_embeddings/clip_scannet_labels_768.pt


inside for loop
tensor([[  0,   0, 322, 150],
        [  0,   1, 324, 150],
        [  0,   1, 326, 151],
        ...,
        [  0, 432, 181,  21],
        [  0, 432, 183,  20],
        [  0, 432, 176,  22]], dtype=torch.int32)
inside for loop
tensor([[  0,   0, 271,  61],
        [  0,   0, 271,  63],
        [  0,   0, 272,  61],
        ...,
        [  0, 447, 183, 119],
        [  0, 447, 256,  69],
        [  0, 447, 256,  56]], dtype=torch.int32)
inside for loop
tensor([[  0,  27, 209,  57],
        [  0,  27, 209,  60],
        [  0,  27, 209,  62],
        ...,
        [  0, 410, 194, 125],
        [  0, 410, 198, 126],
        [  0, 410, 203, 127]], dtype=torch.int32)
inside for loop
tensor([[  0,   0,  88, 123],
        [  0,   0,  88, 124],
        [  0,   0,  85, 122],
        ...,
        [  0, 375, 290,  65],
        [  0, 375, 304, 131],
        [  0, 375, 304, 143]], dtype=torch.int32)
inside for loop
tensor([[  0,   0, 226,  95],
        [  0,   0, 224,  95],
        

RPly: Unable to create file
RPly: Unable to create file
RPly: Unable to create file
[2023-07-01 10:04:08,951 3569469931.py line 6] ==>Training done!
Best Iou: 0.000
[2023-07-01 10:04:08,962 1544885347.py line 18] Load pre-computed embeddings from saved_text_embeddings/clip_scannet_labels_768.pt


[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/openseg_0.ply
[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/pred_0.ply
[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/gt_0.ply
inside for loop
tensor([[  0,   0, 242,  96],
        [  0,   0, 242,  98],
        [  0,   0, 240,  98],
        ...,
        [  0, 437, 295,  73],
        [  0, 437, 294,  71],
        [  0, 437, 288, 132]], dtype=torch.int32)
inside for loop
tensor([[  0,  19, 244, 117],
        [  0,  19, 239,  65],
        [  0,  19, 238,  64],
        ...,
        [  0, 402, 273,   9],
        [  0, 402, 273,  27],
        [  0, 402, 273,  17]], dtype=torch.int32)
inside for loop
tensor([[  0, 415, 211, 159],
        [  0, 415, 213, 159],
        [  0, 415, 128,  12],
        ...,
        [  0,  32, 310,  97],
        [  0,  32, 305,  68],
        [  0, 

RPly: Unable to create file
RPly: Unable to create file
RPly: Unable to create file
[2023-07-01 10:04:11,695 3569469931.py line 6] ==>Training done!
Best Iou: 0.000
[2023-07-01 10:04:11,699 1544885347.py line 18] Load pre-computed embeddings from saved_text_embeddings/clip_scannet_labels_768.pt


...

[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/openseg_8.ply
[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/pred_8.ply
[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/gt_8.ply
inside for loop
tensor([[  0,  13, 192,  41],
        [  0,  13, 192,  62],
        [  0,  13, 193,  42],
        ...,
        [  0, 396, 261,  91],
        [  0, 396, 261,  92],
        [  0, 396, 258,  94]], dtype=torch.int32)
inside for loop
tensor([[  0, 479, 236, 106],
        [  0, 479, 236, 114],
        [  0, 479, 236, 125],
        ...,
        [  0,  32, 176, 133],
        [  0,  32, 179, 133],
        [  0,  32, 179,   7]], dtype=torch.int32)
inside for loop
tensor([[  0, 350, 253,  97],
        [  0, 350, 255,  97],
        [  0, 350, 250,  98],
        ...,
        [  0,  32,  37,   7],
        [  0,  32,  37,   2],
        [  0, 

RPly: Unable to create file
RPly: Unable to create file
RPly: Unable to create file
[2023-07-01 10:04:33,635 3569469931.py line 6] ==>Training done!
Best Iou: 0.000
[2023-07-01 10:04:33,640 1544885347.py line 18] Load pre-computed embeddings from saved_text_embeddings/clip_scannet_labels_768.pt


[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/openseg_9.ply
[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/pred_9.ply
[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/gt_9.ply
inside for loop
tensor([[  0, 412, 237,  85],
        [  0, 412, 236,  84],
        [  0, 411, 236,  83],
        ...,
        [  0,  32, 258, 125],
        [  0,  32, 258, 124],
        [  0,  32, 258,  75]], dtype=torch.int32)
inside for loop
tensor([[  0,   0, 315,  66],
        [  0,   0, 315,  71],
        [  0,   0, 316,  64],
        ...,
        [  0, 374, 112, 142],
        [  0, 374, 117,  98],
        [  0, 374, 115,  98]], dtype=torch.int32)
inside for loop
tensor([[  0,  18, 231, 124],
        [  0,  18, 226, 126],
        [  0,  18, 224,  69],
        ...,
        [  0, 401, 263,  18],
        [  0, 401, 272,  11],
        [  0, 

RPly: Unable to create file
RPly: Unable to create file
RPly: Unable to create file
[2023-07-01 10:04:36,431 3569469931.py line 6] ==>Training done!
Best Iou: 0.000
[2023-07-01 10:04:36,437 1544885347.py line 18] Load pre-computed embeddings from saved_text_embeddings/clip_scannet_labels_768.pt


[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/openseg_10.ply
[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/pred_10.ply
[Open3D WARNING] Write PLY failed: unable to open file: /mnt/project/AT3DCV_Data/experiments/result/last/gt_10.ply
inside for loop
tensor([[  0,  14, 192, 120],
        [  0,  14, 196,   9],
        [  0,  14, 197,   9],
        ...,
        [  0, 397, 259,  83],
        [  0, 397, 272, 106],
        [  0, 397, 272, 114]], dtype=torch.int32)
